# RECALL


In [18]:
import polars as pl,pandas as pd,numpy as np
from tqdm import tqdm

- `train` 从2020-09-20到2022-09-20
- shape:(31788324, 5)
- 截取2018-2019的数据
- 把其中后两个月作为测试集

In [133]:
# 加载transactions_train.csv
train = pl.read_csv('transactions_train.csv', has_header=True) # has_header=True 表示有表头,默认为True 

# 将t_dat转换为日期类型
train = train.with_columns(pl.col("t_dat").str.strptime(pl.Date, "%Y-%m-%d"))

print(train.shape)
train.head()

(31788324, 5)


t_dat,customer_id,article_id,price,sales_channel_id
date,str,i64,f64,i64
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",685687003,0.016932,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",685687004,0.016932,2


In [144]:
 #提取2018-2019年数据
train_filtered = train.filter(
    (pl.col("t_dat").dt.year() >= 2019) & (pl.col("t_dat").dt.year() <= 2020)
)

# 按照年份和月份将数据拆分为train_df和test_df
train_df = train_filtered.filter(
    (pl.col("t_dat").dt.year() == 2019) & (pl.col("t_dat").dt.month() <= 1)
    # (pl.col("t_dat").dt.year() == 2019) & (pl.col("t_dat").dt.month() <= 10)
)

test_df = train_filtered.filter(
    (pl.col("t_dat").dt.year() == 2019) & (pl.col("t_dat").dt.month() >=2) & (pl.col("t_dat").dt.month() <3)
)

# 打印结果
print(f"Train DataFrame Shape: {train_df.shape}")
print(f"Test DataFrame Shape: {test_df.shape}")

train_df['t_dat'].min(),train_df['t_dat'].max(),test_df['t_dat'].min(),test_df['t_dat'].max()


Train DataFrame Shape: (1263471, 5)
Test DataFrame Shape: (1152412, 5)


(datetime.date(2019, 1, 1),
 datetime.date(2019, 1, 31),
 datetime.date(2019, 2, 1),
 datetime.date(2019, 2, 28))

## 1. Most Hot Items
根据一周内全局交易次数，统计交易最多的商品排名

`sub_sample`中的customer_id和train完全一致（unqiue()）,不存在user cold start

In [150]:
# j计算train_last_week:train_df最后一周截断
train_last_week = train_df.filter(pl.col("t_dat").ge(train_df['t_dat'].max() - pl.duration(days=7)))

# 按article_id分组统计交易次数，并按count和t_dat排序
hot_items = train_last_week.group_by('article_id').agg([
    pl.count().alias('count'),
    pl.col('t_dat').max().alias('last_date')
]).sort(
    by=['count', 'last_date'],
    descending=True
)

# 对hot_items进行排序，得到rank
hot_items = hot_items.with_columns(pl.col('count').rank(method='dense', descending=True).alias('rank'))
hot_items.head()


C:\Users\yhb\AppData\Local\Temp\ipykernel_16812\4121463824.py:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias('count'),


article_id,count,last_date,rank
i64,u32,date,u32
689109001,1057,2019-01-31,1
706016001,823,2019-01-31,2
692930001,782,2019-01-31,3
519583015,694,2019-01-31,4
399256013,684,2019-01-31,5


## 2. Co-visitation

- 对每个商品i，如果用户u在时间窗口内购买过i并且购买过j，则认为i和j共现
- 对每个商品i，只保留共现次数最多的前topk个商品，以及对应的共现次数count
- 最终得到每个商品i的共现商品ids和共现次数counts（统计全局不同用户，比如商品i和j在用户u1共现2次，用户u2共现1次，则i和j共现次数为3次）


In [258]:
from datetime import timedelta
from collections import defaultdict

def create_co_visitation(df, time_window=1, topk=10):
    """
    构建基于时间窗口限制的商品共现矩阵,每个商品只保留共现次数最多的前k个商品
    参数:
    - df: 包含交易记录的 Polars DataFrame，必须包含 `customer_id`, `article_id`, `t_dat` 列
    - time_window: 共现的时间窗口，单位：天
    - topk: 每个商品保留的共现商品数量，默认为3
    返回:
    - 共现矩阵字典，格式为 {article_id: {"ids": [topk_co_article_ids], "counts": [topk_co_counts]}}
    """
    # 确保 `t_dat` 列是日期类型
    # df = df.with_columns(pl.col('t_dat').str.strptime(pl.Date, "%Y-%m-%d"))
    
    # 初始化共现矩阵字典
    co_occurrence_dict = defaultdict(lambda: {"ids": [], "counts": []})

    # 按 customer_id 分组，并获取每个用户在购买时间点的数据
    pairs = (
        df.group_by("customer_id")
        .agg([pl.col("article_id").alias("articles"), pl.col("t_dat").alias("dates")])
    )
    
    # 遍历每个用户的购买记录
    for row in pairs.iter_rows():
        articles = row[1]
        purchase_dates = row[2]
        
        # 去重，避免同一商品多次购买
        unique_articles = list(dict.fromkeys(articles))  # 去重后的商品列表
        unique_dates = [purchase_dates[articles.index(article)] for article in unique_articles]  # 对应的日期列表
        
        # 遍历每个购买时间点
        for i in range(len(unique_articles)):
            current_article = unique_articles[i]
            current_date = unique_dates[i]
            
            # 计算该时间点的时间窗口范围
            window_start = current_date - timedelta(days=time_window)
            window_end = current_date + timedelta(days=time_window)

            # 在该时间窗口内找出该用户购买的其他商品
            for j in range(len(unique_articles)):
                if i != j:
                    other_article = unique_articles[j]
                    other_date = unique_dates[j]
                    
                    # 如果其他商品的购买时间在时间窗口内
                    if window_start <= other_date <= window_end:
                        # 确保不重复统计共现
                        for a1, a2 in [(current_article, other_article), (other_article, current_article)]:
                            if a2 not in co_occurrence_dict[a1]["ids"]:
                                co_occurrence_dict[a1]["ids"].append(a2)
                                co_occurrence_dict[a1]["counts"].append(1)
                            else:
                                idx = co_occurrence_dict[a1]["ids"].index(a2)
                                co_occurrence_dict[a1]["counts"][idx] += 1

    # 对每个商品只保留共现次数最多的前topk个商品
    for aid in list(co_occurrence_dict.keys()):
        if len(co_occurrence_dict[aid]["ids"]) > topk:
            # 根据counts排序获取前topk个索引
            topk_indices = sorted(range(len(co_occurrence_dict[aid]["counts"])), 
                                   key=lambda k: co_occurrence_dict[aid]["counts"][k], 
                                   reverse=True)[:topk]
            # 只保留前topk个商品的ids和counts
            co_occurrence_dict[aid]["ids"] = [co_occurrence_dict[aid]["ids"][i] for i in topk_indices]
            co_occurrence_dict[aid]["counts"] = [co_occurrence_dict[aid]["counts"][i] for i in topk_indices]

    return co_occurrence_dict

co_visitation = create_co_visitation(train_df, time_window=2, topk=20)

# 获取商品 1 的共现商品和共现次数
first_key = next(iter(co_visitation))
print(f"First article ID: {first_key}")
print(f"Co-visitation data: {co_visitation[first_key]}")


First article ID: 717779004
Co-visitation data: {'ids': [717779002, 717779001, 689109001, 692930001, 738224001, 712398001, 741215001, 733361001, 708352005, 717779005, 693246005, 708352003, 712406005, 708352004, 697054004, 734137002, 685848003, 639192002, 712587003, 699075002], 'counts': [48, 36, 28, 26, 20, 18, 16, 16, 14, 14, 14, 12, 12, 12, 12, 12, 12, 12, 10, 10]}


In [85]:
# from datetime import timedelta
# from collections import defaultdict

# def create_co_visitation(df, time_window=1, topk=10):
#    """
#    构建基于时间窗口限制的商品共现矩阵,每个商品只保留共现次数最多的前k个商品
#    参数:
#    - df: 包含交易记录的 Polars DataFrame，必须包含 `customer_id`, `article_id`, `t_dat` 列
#    - time_window: 共现的时间窗口，单位：天
#    - topk: 每个商品保留的共现商品数量，默认为3
#    返回:
#    - 共现矩阵字典，格式为 {article_id: {"ids": [topk_co_article_ids], "counts": [topk_co_counts]}}
#    """
#    # 确保 `t_dat` 列是日期类型
# #    df = df.with_columns(pl.col("t_dat").str.strptime(pl.Date, "%Y-%m-%d"))
#    # 获取最新日期
#    current_date = df.select(pl.col("t_dat").max()).item()
#    # 计算时间窗口内的有效交易记录
#    df = df.filter(pl.col("t_dat") >= (current_date - timedelta(days=time_window)))
#    # 按 customer_id 分组，并获取每个用户在时间窗口内的商品列表
#    pairs = (
#        df.group_by("customer_id")
#        .agg([pl.col("article_id").alias("articles")])
#    )
#    # 初始化共现矩阵字典
#    co_occurrence_dict = defaultdict(lambda: {"ids": [], "counts": []})
   
#    # 遍历每个 customer_id 对应的商品列表
#    for row in pairs.rows():
#        articles = row[1]
#        # 去重，避免同一用户同一商品多次购买的影响
#        articles = list(dict.fromkeys(articles))
       
#        for i in range(len(articles)):
#            for j in range(i + 1, len(articles)):
#                aid1, aid2 = articles[i], articles[j]
#                # 确保不统计自身共现，并保持对称性统计
#                if aid1 != aid2:
#                    for a1, a2 in [(aid1, aid2), (aid2, aid1)]:
#                        if a2 not in co_occurrence_dict[a1]["ids"]:
#                            co_occurrence_dict[a1]["ids"].append(a2)
#                            co_occurrence_dict[a1]["counts"].append(1)
#                        else:
#                            idx = co_occurrence_dict[a1]["ids"].index(a2)
#                            co_occurrence_dict[a1]["counts"][idx] += 1

#    # 对每个商品只保留共现次数最多的前topk个商品
#    for aid in list(co_occurrence_dict.keys()):
#        if len(co_occurrence_dict[aid]["ids"]) > topk:
#            # 根据counts排序获取前topk个索引
#            topk_indices = sorted(range(len(co_occurrence_dict[aid]["counts"])), 
#                                key=lambda k: co_occurrence_dict[aid]["counts"][k], 
#                                reverse=True)[:topk]
#            # 只保留前topk个商品的ids和counts
#            co_occurrence_dict[aid]["ids"] = [co_occurrence_dict[aid]["ids"][i] for i in topk_indices]
#            co_occurrence_dict[aid]["counts"] = [co_occurrence_dict[aid]["counts"][i] for i in topk_indices]
   
#    return co_occurrence_dict

# co_visitation = create_co_visitation(train_df,time_window=3, topk=10)

# # Get first item from co_visitation1 dictionary
# first_key = next(iter(co_visitation))
# print(f"First article ID: {first_key}")
# print(f"Co-visitation data: {co_visitation[first_key]}")
# # 
# # print(co_visitation1)

First article ID: 592286002
Co-visitation data: {'ids': [596472002, 548659001, 623434012, 558986004, 661306001, 623347003, 569847002, 573856001, 645065012, 625939004], 'counts': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}


### `recall_by_co_visitation`
根据时间衰竭对最近购买行为的商品进行加权,然后用最近lastn商品分别进入共现矩阵进行查询

假设用户最近购买了3个商品：
- 最近购买: A (权重1.0) -> 共现商品 [X:10次, Y:5次]
- 次近购买: B (权重0.7) -> 共现商品 [X:8次, Z:6次]
- 最早购买: C (权重0.49) -> 共现商品 [Y:7次, Z:4次]

最终推荐分数计算：
```
商品X得分 = 10*1.0 + 8*0.7 = 15.6
商品Y得分 = 5*1.0 + 7*0.49 = 8.43
商品Z得分 = 6*0.7 + 4*0.49 = 6.16
```

推荐顺序：X > Y > Z

In [259]:
def recall_by_co_visitation(user_id, train_df, co_visitation_dict, last_n=3, n_recall=12):
    """
    基于共现矩阵的召回函数
    ...
    """
    # 获取用户最近购买的商品及其购买时间
    user_recent = (
        train_df
        .filter(pl.col('customer_id') == user_id)
        .sort('t_dat', descending=True)
        .select(['article_id', 't_dat'])
        .unique()
        .head(last_n)
    )
    
    # 检查用户是否有购买记录
    if user_recent.is_empty():
        return []  # 返回空列表，表示没有推荐

    user_recent_items = user_recent['article_id'].to_list()
    
    # 存储推荐结果和分数
    recall_items = defaultdict(float)
    
    # 基于用户最近购买的每个商品进行推荐
    weight = 1.0  # 初始权重为1.0
    for idx, item in enumerate(user_recent_items):
        if item in co_visitation_dict:
            # 获取共现商品
            co_items = co_visitation_dict[item]['ids']
            co_counts = co_visitation_dict[item]['counts']
            
            # 为每个共现商品增加分数
            for co_item, count in zip(co_items, co_counts):
                recall_items[co_item] += count * weight
        
        # 下一个商品的权重衰减30%
        weight *= 0.7
    
    # 排序并获取top N推荐结果
    sorted_items = sorted(recall_items.items(), key=lambda x: x[1], reverse=True)
    recommend_items = [item[0] for item in sorted_items[:n_recall]]
    
    # 保存召回结果每个商品的得分
    recommend_items_scores = {item[0]: round(item[1],3) for item in sorted_items[:n_recall]}
    
    return recommend_items, recommend_items_scores


# 使用示例
user_id = train_df['customer_id'][10]  # train2的第一个customerid
recommended_items, recommended_items_scores = recall_by_co_visitation(user_id, train_df, co_visitation, last_n=3, n_recall=12)
print(f"为用户 {user_id} 推荐的商品:")
print(recommended_items)
print(f"推荐商品的得分:")
print(recommended_items_scores)

为用户 00497856651e5483f26917703886e95c6e299a7492874087ec4a2d78acc4f081 推荐的商品:
[399256018, 399256032, 399256030, 399256023, 399256013, 399256002, 636323001, 554450001, 519583028, 706016001, 399256005, 519583008]
推荐商品的得分:
{399256018: 172.0, 399256032: 122.0, 399256030: 102.0, 399256023: 80.0, 399256013: 64.0, 399256002: 60.0, 636323001: 58.0, 554450001: 50.82, 519583028: 38.22, 706016001: 30.0, 399256005: 28.0, 519583008: 27.44}


# Preds on Test

In [212]:
# 查看sub的customer_id和train有多少相同
common_customers = (set(test_df['customer_id']).intersection(set(train_df['customer_id'])))
common_customers = list(common_customers)
print(f"Number of common customers between test and train: {len(common_customers)}")
print(f'unique cid in train: {len(train_df["customer_id"].unique())}')
print(f'unique cid in test: {len(test_df["customer_id"].unique())}')

# sample_submission的customer_id和train完全相同

Number of common customers between test and train: 91634
unique cid in train: 242024
unique cid in test: 238992


In [263]:
def multi_recall(df, user_id, co_visitation, n_recall, topk):
    # 使用covisitation通道召回n_recall个商品
    co_recall_items, co_recall_items_scores = recall_by_co_visitation(user_id, df, co_visitation, last_n=3, n_recall=n_recall)
    # print(co_recall_items_scores)

    # 归一化co_recall_items_scores到0-1之间
    if co_recall_items_scores:  # Check if co_recall_items_scores is not empty
        min_score = min(co_recall_items_scores.values())
        max_score = max(co_recall_items_scores.values())
        co_recall_items_scores = {item: (score - min_score) / (max_score - min_score) for item, score in co_recall_items_scores.items()}
    co_recall_items = list(co_recall_items) 
    print(co_recall)
    
    # 使用hotitem召回
    hot_recall= hot_items[['article_id', 'rank']][:n_recall]  # This is a DataFrame
    hot_recall_items = list(hot_recall['article_id'])
    # 计算hot_recall的得分为rank的导数
    hot_recall_items_scores = {item: 1 / rank for item, rank in zip(hot_recall['article_id'], hot_recall['rank'])}


    # 加权排序
    combined_scores = {}
    for item in co_recall_items + hot_recall_items:
        co_score = co_recall_items_scores.get(item, 0) if isinstance(co_recall_items_scores, dict) else 0
        hot_score = hot_recall_items_scores.get(item, 0) if isinstance(hot_recall_items_scores, dict) else 0
        print(co_score,hot_score)
        if item in co_recall_items and item in hot_recall_items:
            print("both")
            combined_scores[item] = 0.8 * co_score + 0.2 * hot_score
        elif item in co_recall_items:
            print('only co')
            combined_scores[item] = 0.8 * co_score
        elif item in hot_recall_items:
            combined_scores[item] = 0.2 * hot_score

    # 根据加权分数排序
    sorted_items = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    topk_items = [item[0] for item in sorted_items[:topk]]
    topk_items_scores = {item[0]: round(item[1], 3) for item in sorted_items[:topk]}

    # 创建一个DataFrame来展示召回的商品和总得分
    recall_df_hot_covisit = pd.DataFrame({'Item ID': topk_items, 'Total Score': topk_items_scores.values()})
    return topk_items, topk_items_scores, recall_df_hot_covisit

# 使用示例
user_id = common_customers[2]  # train2的第一个customerid
n_recall = 100
topk = 12
recall_items, recall_items_scores, recall_df = multi_recall(train_df, user_id, co_visitation, n_recall, topk)
print(f"为用户 {user_id} 推荐的商品:")
print(recall_items)
print(f"推荐商品的得分:")
print(recall_items_scores)
print(recall_df)

([], {})
1.0 0.041666666666666664
both
0.15384615384615385 0
only co
0.07692307692307693 0
only co
0.07692307692307693 0
only co
0.07692307692307693 0
only co
0.0 0
only co
0.0 0.3333333333333333
both
0.0 1.0
both
0.0 0
only co
0.0 0
only co
0.0 0.0625
both
0.0 0
only co
0.0 0
only co
0.0 0
only co
0.0 0
only co
0.0 0.01639344262295082
both
0.0 0
only co
0.0 0.013513513513513514
both
0.0 0.019230769230769232
both
0.0 0.03333333333333333
both
0.0 1.0
both
0 0.5
0.0 0.3333333333333333
both
0 0.25
0 0.2
0 0.16666666666666666
0 0.14285714285714285
0 0.125
0 0.1111111111111111
0 0.1
0 0.09090909090909091
0 0.08333333333333333
0 0.07692307692307693
0 0.07142857142857142
0 0.06666666666666667
0.0 0.0625
both
0 0.058823529411764705
0 0.05555555555555555
0 0.05263157894736842
0 0.05
0 0.047619047619047616
0 0.045454545454545456
0 0.043478260869565216
1.0 0.041666666666666664
both
0 0.04
0 0.038461538461538464
0 0.037037037037037035
0 0.03571428571428571
0 0.034482758620689655
0.0 0.033333333333

In [241]:
user_id = common_customers[2]
co_recall = recall_by_co_visitation(user_id, train_df, co_visitation, last_n=3, n_recall=n_recall)

hot_recall= hot_items[['article_id', 'rank']][:n_recall]  # This is a DataFrame
hot_recall_items = list(hot_recall['article_id'])
    # 计算hot_recall的得分为rank的导数
hot_recall_items_scores = {item: 1 / rank for item, rank in zip(hot_recall['article_id'], hot_recall['rank'])}


hot_recall_items_scores.get(hot_recall_items[1],0)

0.5